<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

홍종선(2009)에서는 의미적 차원과 표현 구조를 검토하여 감정동사들을 총19유형(감동, 감사, 기쁨, 고뇌, 고독, 놀람, 동정, 두려움, 미안함, 분노, 상쾌함,수용, 수치심, 슬픔, 욕구, 이완, 허무, 혐오, 후회)으로 분류했다. 표현 구조를고려하여 분류된 감정동사들의 벡터 값이 유사할 것이라는 가정 하에 Word2Vec을 활용하여 이를 검토해보고자 한다. 

데이터는 AI 허브에서 제공하는 감성분석 데이터에서 content만 따로 추출하여 데이터로 사용한다.

In [1]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from gensim.models import word2vec

In [2]:
text = codecs.open("sent.content.txt", "r", encoding = "utf-8")

In [3]:
f = open("sent.content.txt", "r", encoding="utf-8")
print(f.read(20))

곧 만기 은퇴 앞두다 있다 노후 준비


In [4]:
word_dict = {}
lines = f.read().split("\n")
print(lines[10])

친구 내 자다 되다 건 다 너 덕분 고맙다 문자 하다 통 넣다


In [5]:
morph_analyzer = Okt()
morph_analyzer1 = Okt()
for line in lines:
    morph_analysed = morph_analyzer.pos(line)
    for word in morph_analysed:
        if word[1] == "Noun":
            if not (word[0] in word_dict):
                word_dict[word[0]] = 0
            word_dict[word[0]] += 1    

C:\Users\yein4\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [7]:
keys = sorted(word_dict.items(), key=lambda x:x[1], reverse=True)

In [8]:
print(keys[:10])

[('것', 7900), ('수', 6570), ('친구', 5653), ('내', 5178), ('나', 3815), ('상황', 3586), ('기분', 3182), ('더', 3182), ('생각', 3003), ('말', 2977)]


In [9]:
for word, count in keys[:20]:
    print("{0}{1}".format(word, count))
    
f.close()

것7900
수6570
친구5653
내5178
나3815
상황3586
기분3182
더3182
생각3003
말2977
마음2867
지금2799
안2658
일2477
이2434
못2432
사람2352
게2351
거2128
정말2059


In [ ]:
with open("sent.content.txt", "r", encoding="utf-8") as fr:
    results = []
    lines = fr.read().split("\n")
    for line in lines:
        morph_analysed = morph_analyzer.pos(line, norm=True, stem=True)
        # stem=True -> 어근으로 출력하라는 의미
        # ex) '그래요' -> '그렇다'
        #print(mallist)
        r = []
        for word in morph_analysed:
            if not word[1] in ["Josa", "Punctuation", "Foreign", "Suffix", "Eomi"]:
                r.append(word[0])
            # 결과에서 제외 할 품사 입력하기
        #print(r) 
        rl = (" ".join(r)).strip() #공백제거
        results.append(rl)        

In [33]:
print(results[:4])

['곧 만기 은퇴 앞두다 있다 노후 준비 미리 꼼꼼하다 해 두 어서 기쁘다', '노후 준비 미리 꼼꼼하다 해 두 어서 기쁘다', '이렇게 노후 준비 하다 수 있다 건 옆 도와주다 친구 덕분 이 고마움 뭐 갚다', '노후 준비 도운 친구 고마움 느끼다 있다 상대방 고마움 표현 하다 수 있다 좋다 방법 있다']


In [11]:
data_file = "sent.content.txt"
with open(data_file, "w", encoding="utf-8") as fw:
    fw.write("\n".join(results))
           
from gensim.models import word2vec
data = word2vec.LineSentence(data_file)
print(data)
model = word2vec.Word2Vec(data, size=100, window=10, hs=1, min_count=2, sg=1)
# CBOW, Skip-gram(0)
model.init_sims(replace=True) #필요없는 메모리는 unload
#model.save("gimal.model")
print("done")

done


In [12]:
##model = word2vec.Word2Vec.load("gimal.model")
##print(model.similarity("기쁘다", "슬프다")) 
##print(model.similarity("기쁘다", "행복하다")) 

In [13]:
print(model.most_similar("기쁘다"))

[('행복하다', 0.6797296404838562), ('뿌듯하다', 0.6624724864959717), ('자랑스럽다', 0.6465882062911987), ('값지다', 0.6333721280097961), ('다행', 0.6318965554237366), ('최고', 0.6219849586486816), ('센스', 0.6149460077285767), ('만족스럽다', 0.6145679950714111), ('예식', 0.6099244356155396), ('축하', 0.5977829098701477)]


C:\Users\yein4\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [14]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format("gimal_model")

In [15]:
# PCA 돌리기 위해 모델 텐서를 저장한다
#!python -m gensim.scripts.word2vec2tensor --input gimal_model --output gimal_model

차원 축소한 벡터를 통해 감정동사에 해당하는 동사들의 cosine distance를확인하였다. cosine distance는 1-cosine similarity로, 값이 작을수록 유사도가 높은 것이다. 각 동사들의 cosine distance 값은 다음과 같다:

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
data=pd.read_excel('./gimal.xlsx')
pd.set_option('display.max_rows',500)
data

,기준동사분류,기준동사,관련동사분류,관련동사,CD
0,감동,NaN,NaN,NaN,NaN
1,감사,고맙다,기쁨,기쁘다,0.544
2,감사,고맙다,미안함,미안하다,0.518
3,고뇌,걱정스럽다,고뇌,불안하다,0.531
4,고뇌,곤란하다,고뇌,난감하다,0.469
5,고뇌,괴롭다,고뇌,불안하다,0.545
6,고뇌,괴롭다,고뇌,착잡하다,0.569
7,고뇌,괴롭다,슬픔,슬프다,0.464
8,고뇌,괴롭다,혐오,속상하다,0.505
9,고뇌,괴롭다,두려움,무섭다,0.521


In [18]:
v=data['기준동사분류'].value_counts()
v2=v.to_frame()
#v2.to_excel('gimal_v2.xlsx')
v2

,기준동사분류
슬픔,57
혐오,43
고뇌,28
기쁨,24
동정,19
고독,15
수치심,14
미안함,12
두려움,12
수용,9


In [19]:
size1=data.groupby(['기준동사분류','관련동사분류']).size()
size2=size1.to_frame()
size2

0
기준동사분류 관련동사분류    
감사     기쁨       1
       미안함      1
고뇌     고뇌       8
       놀람       1
       동정       2
       두려움      5
       슬픔       6
       허무       1
       혐오       5
고독     고독       3
       동정       1
       두려움      1
       슬픔       9
       혐오       1
기쁨     감사       1
       기쁨      13
       수용      10
놀람     고뇌       1
       놀람       2
       슬픔       1
       혐오       1
동정     고뇌       2
       고독       2
       동정       2
       슬픔      10
       허무       1
       혐오       2
두려움    고뇌       5
       고독       1
       두려움      2
       미안함      1
       슬픔       3
미안함    감사       1
       두려움      1
       미안함      4
       슬픔       1
       혐오       5
복합감정   상쾌함      1
분노     슬픔       1
       혐오       3
상쾌함    고뇌       1
       복합감정     1
       상쾌함      5
       슬픔       1
       허무       1
수용     기쁨       9
수치심    수치심      5
       슬픔       3
       허무       2
       혐오       4
슬픔     고뇌       4
       고독       9
       동정       5
       분노       1
       상쾌함      2
       수치심      4
       슬픔      21
       허무       1
       혐오      10
욕구     혐오       2
허무     고뇌       1
       동정       1
       수치심      2
       슬픔       1
       허무       2
       혐오       1
혐오     고뇌       4
       고독       1
       놀람       1
       동정       2
       미안함      3
       분노       2
       수치심      4
       슬픔      11
       허무       1
       혐오      13
       후회       1
후회     혐오       1

18개이 범주 중 10개의 범주에서 기준동사범주와 관련동사범주의 일치율이 높은 것으로 확인되는데, 이를 통해 홍종선(2009)에서 제시된 감정동사의 분류 기재와 결과가 유의미하다는 것을 알 수 있다. 또한 각 기준동사범주에 대해 특정 관련동사범주의 비율이 높게 나옴을 확인할 수 있는데, 이는 감정동사범주들 간의 유사성도 존재한다는 것을 의미한다. 따라서 이후 규모가 더 큰 데이터를 사용하여 범주들 간의 유사성을 검증한 후 이를 다시 범주화할 수 있을 것으로 보인다. 이렇게 범주를 다시 범주화 하는 것은 세밀하게 나눠진 인간 감정의 범주를 덜 세분화하여 컴퓨터가 인간이 구사한 문장 속의 감정동사를 통해 인간이 전달하고자 한 감정과 근접한 감정을 인식할 수 있는또 다른 방법이 될 수도 있을 것이라 생각된다.